<a href="https://colab.research.google.com/github/mahaksinhal/merit-list-and-scholarship-allocation/blob/main/LEX_and_YACC_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEX and YACC Compiler in Colab

Drawbacks:
* Regular interrupts (Ctrl+D, Ctrl+C) for shell won't work in Colab while inputting for program.
<br>Workaround: Store your inputs in a txt file and pass it to the program.

In [1]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bison is already the newest version (2:3.8.2+dfsg-1build1).
flex is already the newest version (2.6.4-8build2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


## Lex only

In [2]:
#@title Writing Lex program
%%writefile dag_edge.l
%{
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define MAX 50

// Node structure for DAG
struct Node {
    char op;
    char left[10];
    char right[10];
    int index;
} dag[MAX];

int node_count = 0;

// === Function Declarations ===
int search(char op, char *left, char *right);
void insert(char op, char *left, char *right);
void display_nodes();
void display_edges();
void trim(char *str);

%}

%option noyywrap

%%
[ \t\n]       ;
[a-zA-Z]      { printf("Identifier: %s\n", yytext); }
"+"|"-"|"*"|"/" {
                    char op = yytext[0];
                    printf("Operator: %c\n", op);
                }
.              ;
%%

// === Helper and DAG Functions ===

void trim(char *str) {
    char *p1 = str, *p2 = str;
    while(*p2 != '\0') {
        if(*p2 != ' ')
            *p1++ = *p2;
        p2++;
    }
    *p1 = '\0';
}

int search(char op, char *left, char *right) {
    for (int i = 0; i < node_count; i++) {
        if (dag[i].op == op &&
            strcmp(dag[i].left, left) == 0 &&
            strcmp(dag[i].right, right) == 0)
            return i;
    }
    return -1;
}

void insert(char op, char *left, char *right) {
    int pos = search(op, left, right);
    if (pos == -1) {
        dag[node_count].op = op;
        strcpy(dag[node_count].left, left);
        strcpy(dag[node_count].right, right);
        dag[node_count].index = node_count + 1;
        node_count++;
    }
}

void display_nodes() {
    printf("\n--- DAG Nodes ---\n");
    printf("Index\tOp\tLeft\tRight\n");
    for (int i = 0; i < node_count; i++) {
        printf("%d\t%c\t%s\t%s\n", dag[i].index, dag[i].op, dag[i].left, dag[i].right);
    }
}

void display_edges() {
    printf("\n--- DAG Edges (Parent → Child) ---\n");
    for (int i = 0; i < node_count; i++) {
        printf("Node%d(%c) → %s\n", dag[i].index, dag[i].op, dag[i].left);
        printf("Node%d(%c) → %s\n", dag[i].index, dag[i].op, dag[i].right);
    }
}

int main() {
    char expr[100];
    printf("Enter an expression (e.g., a+b*c+b*c): ");
    scanf("%s", expr);

    trim(expr);

    // Simple parsing: handle left-to-right binary operators
    for (int i = 0; i < strlen(expr); i++) {
        if (expr[i] == '+' || expr[i] == '-' || expr[i] == '*' || expr[i] == '/') {
            char left[10] = { expr[i - 1], '\0' };
            char right[10] = { expr[i + 1], '\0' };
            insert(expr[i], left, right);
        }
    }

    display_nodes();
    display_edges();

    printf("\n--- Lexical Analysis ---\n");
    yylex();

    return 0;
}


Writing dag_edge.l


In [3]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

lex dag_edges.l
gcc lex.yy.c -o dag_edges
./dag_edges


/bin/bash: line 2: !flex: command not found
/bin/bash: line 3: !gcc: command not found
/bin/bash: line 4: !echo: command not found

--- DAG Representation ---
Index	Op	Left	Right
1	+	a	b
2	*	b	c
--------------------------
